In [7]:
!pip install tensorflow tensorflow-datasets


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import numpy as np

# Load dataset via TFDS
(ds_train, ds_val), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_classes = ds_info.features['label'].num_classes

IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Build model with EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# Train
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10
)

# Optional fine‑tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
fine_history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)

# Calorie mapping (populate all 101 classes)
class_names = ds_info.features['label'].names
calorie_dict = {name: 300 for name in class_names}  # replace with real values

# Prediction + calorie function
def predict_food_and_calories(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    arr = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    arr = tf.expand_dims(arr, 0)
    preds = model.predict(arr)
    idx = np.argmax(preds)
    name = class_names[idx]
    confidence = float(np.max(preds)) * 100
    calories = calorie_dict.get(name, "Unknown")
    return name, calories, confidence

# Example usage:
# food, cal, conf = predict_food_and_calories("my_food_image.jpg")
# print(f"{food} ({conf:.1f}%): {cal} kcal")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        25,957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,403,464 (16.80 MB)

 Trainable params: 353,893 (1.35 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 270s 103ms/step - accuracy: 0.0096 - loss: 4.6200 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 2/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 195s 81ms/step - accuracy: 0.0102 - loss: 4.6158 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 3/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 231s 97ms/step - accuracy: 0.0092 - loss: 4.6158 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 4/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 193s 80ms/step - accuracy: 0.0088 - loss: 4.6157 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 5/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 191s 80ms/step - accuracy: 0.0077 - loss: 4.6157 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 6/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 233s 97ms/step - accuracy: 0.0093 - loss: 4.6158 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 7/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 193s 81ms/step - accuracy: 0.0100 - loss: 4.6157 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 8/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 233s 97ms/step - accuracy:

In [10]:
model.save("food101_model.keras")

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
macaroni_and_cheese (1.0%): 300 kcal
